In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
import tensorflow_datasets as tfds

# Load dataset
dataset_name = "cats_vs_dogs"
(raw_train, raw_test), metadata = tfds.load(dataset_name, split=['train[:80%]', 'train[80%:]'], as_supervised=True, with_info=True)


In [ ]:
IMG_SIZE = 150  # Resize to 150x150

def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize
    image = image / 255.0  # Normalize
    return image, label

train = raw_train.map(format_image)
test = raw_test.map(format_image)

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
BATCH_SIZE = 32
train_batches = train.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [ ]:
history = model.fit(train_batches, epochs=10, validation_data=test_batches)

In [ ]:
model.save("cats_vs_dogs_best.h5")

In [24]:
test_loss, test_acc = model.evaluate(test_batches)
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")

146/146 ━━━━━━━━━━━━━━━━━━━━ 74s 502ms/step - accuracy: 0.7995 - loss: 0.4905

Test Accuracy: 80.65%


In [25]:
from tensorflow.keras.models import load_model
model = load_model("cats_vs_dogs_best.h5")

import cv2
import numpy as np
img = cv2.imread("/content/88ff5600-d979-11ef-a5c8-1da73bd59591.jpg")  # Load a test image
img = cv2.resize(img, (150, 150))  # Resize to model input shape
img = np.expand_dims(img, axis=0) / 255.0  # Normalize
prediction = model.predict(img)
print("Dog" if prediction > 0.5 else "Cat")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Dog
